<a href="https://colab.research.google.com/github/BioML-UGent/Advanced-AI-for-Bioinformatics/blob/main/06_calibration/PClab_6_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PC-Lab: Calibration of Machine Learning Models


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.datasets import make_classification, make_blobs
from sklearn.naive_bayes import GaussianNB

%matplotlib inline

For this tutorial, we'll make use of the python package [pycalib](https://classifier-calibration.github.io/PyCalib/). It first has to installed into your environment.

In [ ]:
!pip install pycalib
import pycalib

## Introduction to Calibration in Machine Learning

In machine learning, uncertainty estimation is crucial for understanding the confidence level of model predictions.
 It involves quantifying the degree of certainty or doubt associated with a model's predictions.

## What is Calibration?

Calibration in machine learning refers to the process of aligning a model's predicted probabilities with the actual outcomes.
 A well-calibrated model provides probability estimates that match the true likelihood of an event occurring.
 For example, if a calibrated model predicts an event with 70% probability,
then approximately 70% of the predictions at this probability level should be correct.

### The Importance of Calibration

Calibration is essential for two main reasons:

- **Trustworthiness**: A calibrated model ensures _trustworthiness_ in its predictions, which is vital for decision-making processes.
- **Decision-making**: In some applications, decisions are made based on the predicted probabilities rather than the classification itself. Accurate probability estimates are, therefore, crucial.

### Assesing Model Calibration

1. **Calibration Curves**

<img src="https://www.researchgate.net/publication/368304257/figure/fig2/AS:11431281118358760@1675739847164/Example-of-a-calibration-curve-which-plots-the-observed-frequency-of-instances-belonging.ppm" width=400>

2. **Analytical Scores**, i.e. calibration error estimates that estimate confidence, classwise or strong notions of calibration

## 1. Binary dataset

In the first task, we will look at the simple case of binary classification, and compare the calibration of different classifier models
using calibration curves and the expected calibration error as ann error estimate.

In [ ]:
from sklearn.datasets import make_blobs

dataset_binary = make_blobs(n_samples=10000, centers=5, n_features=2, random_state=42)

# make the dataset binary
dataset_binary[1][:] = dataset_binary[1] > 2


Visualise the dataset in a scatterplot:

In [ ]:
x, y = dataset_binary

fig, ax = plt.subplots(figsize=(6, 5))
scatter = ax.scatter(x[:, 0], x[:, 1], c=y, cmap='viridis')
handles, labels = scatter.legend_elements()
labels = ['Class 0', 'Class 1']

legend = ax.legend(handles, labels, loc="upper right", title="Classes")
ax.add_artist(legend)
plt.title('Binary Classification Dataset')
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
plt.show()



### Exercise 1
Split your training data into training and validation, use a bigger percentage (50 %) for testing the calibration afterwards. (Why??). Fit a simple Naive Bayes Classifier on your data.  


In [ ]:
# train test split
x_train, x_test, y_train, y_test = # ....

# classifier
clf = # ....
clf.fit(x_train, y_train)

_Why choose a higher percentage for the test set_?
- higher reliability: more reliable and stable calibration estimates
- generalizability
- statisitical significance: more statistical power to detect differences/issues in calibration

### Exercise 2
Visualize the predicted probabilities of your classifier predicted for class 1 in dependence of the feature space using <a href="https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contour.html" target="_top"> contour lines </a> and histograms. For the latter, use 10 equal-length bins to divide the predicted probabilities.


In [ ]:
# divide the feature space into a grid
delta = 0.25
x0_grid = np.arange(x[:, 0].min(), x[:, 0].max(), delta)
x1_grid = np.arange(x[:, 1].min(), x[:, 1].max(), delta)
X0, X1 = np.meshgrid(x0_grid, x1_grid)
# predict the probabilities for each point on the grid
Y = # ....

fig, ax = plt.subplots(figsize=(6, 5))
# do a contour plot of the probabilities for class 1
CS = # ax.contour(...)
ax.clabel(CS, inline=1, fontsize=15)
ax.scatter(x_test[:1000, 0], x_test[:1000, 1], c=y_test[:1000])
ax.set_title('Predicted probabilities')
ax.grid()

Histogram:

In [ ]:
# divide the scores into bins
n_bins = 10
bins = # ...

# predictions on test set
p_pred = # ...

fig, ax = plt.subplots(figsize=(6, 5))

ax.hist([p_pred[y_test == 0, 1], p_pred[y_test == 1, 1]], bins=bins, orientation='horizontal', label=['Class 0', 'Class 1'],
        color=["purple", "yellow"])
ax.set_ylim(0, 1)
ax.grid()
ax.set_xlabel('Number of samples per bin')
ax.set_ylabel('Predicted probability')

We see that the model gives lower scores to the first class, and that most of the instances of that class
are in the range of scores between $0$ and $0.1$. The last bin between $0.9$ and $1.0$ does not contain any sample.

### Exercise 3
Plot a reliability diagram. In order to do this, use a number of B=10 bins to divide the predicted probabilities Use the <a href="https://numpy.org/doc/stable/reference/generated/numpy.digitize.html" target="_top"> numpy.digitize </a> function to return the index of the corresponding bin for each instance. Compute the mean score for each bin and compare the true positive rate in each bin (i.e. the percentage of instances classified as 1).


In [ ]:
# assign each of the instances to a bin
bin_index = #....

# compute the mean score in each bin
pred_means = # ....

y_means = # ....

Now we can plot a line with the mean scores in the $x$ axis and the true positive rate on the $y$ axis.

In [ ]:
fig, ax = plt.subplots( figsize=(7, 5))
ax.plot(pred_means, y_means, 'o-')
ax.plot([0, 1], [0, 1], 'r--')
ax.set_xlim(0, 1)
ax.set_xlabel('Mean predicted value')
ax.set_ylim(0, 1)
ax.set_ylabel('Fraction of positives')
ax.set_title('Calibration curve for class 1')
ax.grid()

We see that the model is

- _over-confident_ in the lower range of scores: predicts scores of 0.2 when the true proportion of positives is around 10%.
- _under-confident_ in the higher range of scores: predicts scores of 0.8 when the true proportion of positives is 100 %.

### Exercise 4
Train a KNN classifier on the data and compare the calibration to the NaiveBayes in a joint calibration curve plot. Use the pycalib.visualisations functions plot_reliability_diagram and plot_binary_reliability_diagram_gaps.


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf2 = # ...
clf2.fit(#....)

In the PyCalib library we can directly call a function to plot the reliability diagram for a list of output scores. Use the function [```plot_reliability_diagrams```](https://classifier-calibration.github.io/PyCalib/api/visualisations.html#pycalib.visualisations.plot_reliability_diagram) to visualize the calibration curve for both classifiers.

In [ ]:
from pycalib.visualisations import plot_reliability_diagram

scores_list = # ....
labels_list = ['KNN', 'GaussianNB']

# plot the reliability diagram
#
#

We can also compute the expected calibration error fro both classifiers:

In [ ]:
from pycalib.metrics import ECE

ece_list = [ECE(y_test, scores) for scores in scores_list]
print(f"ECE for KNN: {ece_list[0]:.3f}")
print(f"ECE for GaussianNB: {ece_list[1]:.3f}")

## 2. Multiclass Setting

Let's do the same analysis for a multiclass problem.

In [ ]:
n_features = 7
dataset_ternary = make_classification(n_classes=3, n_samples=10000, n_clusters_per_class=3, n_features=n_features, n_informative=5, random_state=42)

# visualize the dataset
x, y = dataset_ternary
fig, ax = plt.subplots(7, 7, figsize=(12, 10))

for i in range(n_features):
    for j in range(n_features):
        ax[i, j].scatter(x[:, i], x[:, j], c=y, cmap='viridis')
        ax[i, j].set_xticks([])
        ax[i, j].set_yticks([])
        ax[i, j].set_xlabel(f'Feature {i}')
        ax[i, j].set_ylabel(f'Feature {j}')

plt.tight_layout()

### Exercise 5
Again train a Naive Bayes classifier and a k-nearest neighbor classifier with n=10 neighbors on your dataset. Plot the reliability curves for both of them. How do you evaluate calibration in the multi-class scenario?



In [ ]:
x_train, x_test, y_train, y_test = # ....

# define and fit classifiers
clf = # ... Naive Bayes
# clf.fit(...)
clf2 = #  k-nearest neighbor
# clf2.fit(...)

scores_list = [clf.predict_proba(x_test),
          clf2.predict_proba(x_test)]

fig = plt.figure(figsize=(10,6 ))

# plot the reliability curve
# _ = plot_reliability_diagram(....)

### Exercise 6
Calculate confidence and classwise ECE for both classifiers using the pre-implemented functions from pycalib.


In [ ]:
from pycalib.metrics import conf_ECE, classwise_ECE

# your code
#
#

## 3. Calibration Maps: Isotonic Regression

In the last section, we will look at two ways of calibrating the predictions of a model after it has been trained.

**Platt Scaling**, also known as logistic calibration, fits a logistic regression model to the classifier's output. In particular,

having a model $f$, the transformation $$P(y=1|x) = \frac{1}{1 + \exp(Af(x) + B)}$$ is used to logistically transform the scores.

On the other hand, **Isotonic Regression** works by fitting a non-decreasing functiion to the model's outputs, which is learned by minimizing a least-squares criterion on the calibration data.



We will load again the binary dataset that we created earlier, and use the Naive BAyes Classifier to compare the two methods.

In [ ]:
x, y = dataset_binary
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

### Exercise 7

This time, use the sklearn class <a href="https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html" target="_top">CalibratedClassifierCV </a> to train three different classifiers: the uncalibrated naive Bayes, the naive Bayes with Platt scaling calibration and the naive Bayes with isotonic regression calibration.



In [ ]:
from sklearn.calibration import CalibratedClassifierCV

clf = # ....   uncalibrated naive Bayes
clf.fit(x_train, y_train)

cal_clf_1 = # ....   calibrated naive Bayes - platt scaling
cal_clf_1.fit(x_train, y_train)

cal_clf_2 = # ....   calibrated naive Bayes - isotonic regression
cal_clf_2.fit(x_train, y_train)




### Exercise 8
Visualize the reliabiltiy diagram of all three classifiers.


In [ ]:
fig = plt.figure(figsize=(10, 6))

# _ = plot_reliability_diagram(...)

#### Comparison of the two calibration maps

Isotonic Regression is closer to the diagonal line.